In [2]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
import os


from tensorflow.keras import datasets
from tensorflow.keras import layers


from tensorflow.keras.models import load_model
model = load_model('my_models.h5')



In [3]:
def order_points(pts):
  
    rect = np.zeros((4, 2), dtype = "float32")

    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]


    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

def ret_result(card_num):
        if(card_num != 52):
            mark = card_num // 13
            number = card_num % 13 + 1
        else:
            mark = 4
        
        if(number == 11):
            number = "J"
        elif(number == 12):
            number = "Q"
        elif(number == 13):
            number = "K"
        elif(number == 1):
            number = "A"
        
        if(mark == 0) :
            return("spade {}".format(number))
        elif(mark == 1) :
            return("clover {}".format(number))
        elif(mark == 2) :
            return("diamond {}".format(number))
        elif(mark == 3) : 
            return("heart {}".format(number))
        else:
            return("unknown")

def predict_img(v):
    
    v = v[tf.newaxis,...]
    pred = model.predict(v)
    card_num = pred.argmax()

    return ret_result(card_num)

In [ ]:
def auto_scan_image_via_webcam():
    
    try: 
        cap = cv2.VideoCapture(0)
    except:
        print ('cannot load camera!')
        return
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print ('cannot load camera!')
            break
            
        k = cv2.waitKey(10)
        if k == 27:
            cap.release()
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.GaussianBlur(gray, (3, 3), 0)
        edged = cv2.Canny(gray, 75, 200)

        print ("STEP 1: Edge Detection")
        
        (cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:6]
        
    
        warpeds = []
        screenCnts = []
       
        for c in cnts:
            
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            screenCnt = []
            
           
            if len(approx) == 4:
                contourSize = cv2.contourArea(approx)
                camSize = frame.shape[0] * frame.shape[1]
                ratio = contourSize / camSize
                print (contourSize)
                print (camSize)
                print (ratio)
                
                if ratio > 0.06:
                    screenCnt = approx
                    screenCnts.append(screenCnt)
                    
        
        if len(screenCnt) == 0:
            cv2.imshow("WebCam", frame)
            continue
            
        else:
           
            for v in screenCnts:
                rect = order_points(v.reshape(4, 2))
                (topLeft, topRight, bottomRight, bottomLeft) = rect

                w1 = abs(bottomRight[0] - bottomLeft[0])
                w2 = abs(topRight[0] - topLeft[0])
                h1 = abs(topRight[1] - bottomRight[1])
                h2 = abs(topLeft[1] - bottomLeft[1])
                maxWidth = max([w1, w2])
                maxHeight = max([h1, h2])

                dst = np.float32([[0,0], [maxWidth-1,0], [maxWidth-1,maxHeight-1], [0,maxHeight-1]])

                M = cv2.getPerspectiveTransform(rect, dst)
                warped = cv2.warpPerspective(frame, M, (maxWidth, maxHeight))
                
                
                w = cv2.resize(warped, dsize = (100, 150) , interpolation=cv2.INTER_AREA)

                w=tf.convert_to_tensor(w)
                
                cv2.drawContours(frame, [v] , -1, (0, 255, 0), 2)
                cv2.putText(frame, str(predict_img(w)), (int((topLeft[0]+topRight[0])/2), int(topRight[1])), cv2.FONT_HERSHEY_PLAIN, 2 ,(0,255,255), 2)
                
                
            cv2.imshow("WebCam", frame)
            
            

                
           
            #break
            


    #cap.release()
    #cv2.destroyAllWindows()
    #cv2.waitKey(1)
    
    '''
    for w in warpeds:
        cv2.imshow("Scanned", w)
        w = cv2.resize(w, dsize = (50, 75) , interpolation=cv2.INTER_AREA)

        w=tf.convert_to_tensor(w)
        tf_warpeds.append(w)
        cv2.waitKey(0)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    '''
    
        

if __name__ == '__main__':
   auto_scan_image_via_webcam()